In [52]:
import time
from pyspark.sql.functions import to_date, sequence, explode, sum, month, year, concat_ws, collect_list,desc

In [2]:
%run 01_setup.ipynb

23/11/19 17:18:39 WARN SSLSocketFactoryEx: Failed to load OpenSSL. Falling back to the JSSE default.


In [3]:
# take only clear data
ex_clear = ex_df.filter(to_date("srch_ci") <= to_date("srch_co"))

In [14]:
%%sparksql
-- take only clear data
CREATE OR REPLACE TEMP VIEW ex_clear AS
    SELECT * FROM ex WHERE date(srch_ci) <= date(srch_co);

In [15]:
%%sparksql
-- sum booked rooms by dates
CREATE OR REPLACE TEMP VIEW visits AS
    WITH v AS (
        SELECT 
            hotel_id,
            explode(sequence(date(srch_ci),date(srch_co), INTERVAL 1 DAY)) AS date,
            srch_rm_cnt
        FROM ex_clear)
    SELECT hotel_id, date, sum(srch_rm_cnt) AS used_rooms FROM v GROUP BY hotel_id, date;

In [19]:
#sum booked rooms by dates
ex_df1 = ex_clear.select('hotel_id', explode(sequence(to_date('srch_ci'),to_date('srch_co'))).alias('date'), 'srch_rm_cnt')
visits_df = ex_df1.groupBy('hotel_id', 'date') \
    .agg(concat_ws("_", year('date'), month('date')).alias('year_month'),sum('srch_rm_cnt').alias('used_rooms'))

In [14]:
for col in visits_df.dtypes:
    print(col[0]+" , "+col[1])

hotel_id , bigint
date , date
FM , string
used_rooms , bigint


In [15]:
visits_df.show(3)

+-------------+----------+------+----------+
|     hotel_id|      date|    FM|used_rooms|
+-------------+----------+------+----------+
| 970662608899|2017-08-22|2017_8|         1|
| 970662608899|2017-08-23|2017_8|         1|
|2405181685760|2017-09-16|2017_9|         1|
+-------------+----------+------+----------+
only showing top 3 rows



In [56]:
list_of_rows = visits_df.filter(visits_df.year_month == '2017_8').orderBy(desc("used_rooms")).take(10)
df = spark.createDataFrame(list_of_rows)
df.show()

+-------------+----------+----------+----------+
|     hotel_id|      date|year_month|used_rooms|
+-------------+----------+----------+----------+
|3384434229251|2017-08-24|    2017_8|         4|
|3384434229251|2017-08-19|    2017_8|         4|
|3384434229251|2017-08-20|    2017_8|         4|
|3384434229251|2017-08-21|    2017_8|         4|
|3384434229251|2017-08-22|    2017_8|         4|
|3384434229251|2017-08-23|    2017_8|         4|
|1889785610243|2017-08-17|    2017_8|         3|
|  94489280513|2017-08-26|    2017_8|         3|
|1889785610243|2017-08-16|    2017_8|         3|
|  94489280513|2017-08-25|    2017_8|         3|
+-------------+----------+----------+----------+



In [59]:
list_of_rows = []
distinct_yms = [x.year_month for x in visits_df.select('year_month').distinct().collect()]
for ym in distinct_yms:
    list_of_rows.extend(visits_df.filter(visits_df.year_month == ym).orderBy(desc("used_rooms")).take(10))
top10_by_month = spark.createDataFrame(list_of_rows)
top10_by_month.show()

+-------------+----------+----------+----------+
|     hotel_id|      date|year_month|used_rooms|
+-------------+----------+----------+----------+
|3384434229251|2017-08-24|    2017_8|         4|
|3384434229251|2017-08-19|    2017_8|         4|
|3384434229251|2017-08-20|    2017_8|         4|
|3384434229251|2017-08-21|    2017_8|         4|
|3384434229251|2017-08-22|    2017_8|         4|
|3384434229251|2017-08-23|    2017_8|         4|
|1889785610243|2017-08-17|    2017_8|         3|
|  94489280513|2017-08-26|    2017_8|         3|
|1889785610243|2017-08-16|    2017_8|         3|
|  94489280513|2017-08-25|    2017_8|         3|
|1889785610247|2017-09-10|    2017_9|         7|
|1889785610247|2017-09-11|    2017_9|         7|
|1889785610247|2017-09-12|    2017_9|         7|
|1443109011462|2017-09-02|    2017_9|         4|
|1443109011462|2017-09-01|    2017_9|         4|
|1443109011462|2017-09-03|    2017_9|         4|
|1443109011462|2017-09-04|    2017_9|         4|
|1872605741056|2017-

In [62]:
top10_by_month.explain()

== Physical Plan ==
*(1) Scan ExistingRDD[hotel_id#476L,date#477,year_month#478,used_rooms#479L]




In [ ]:
%%sparksql
WITH ymCTE(year, month) AS (
    SELECT DISTINCT year(date) AS year, month(date) AS month 
    FROM visits
)
SELECT hotel_id, ymCTE.year, ymCTE.month FROM visits LIMIT 10

In [ ]:
%%sparksql
CREATE OR REPLACE TEMPORARY VIEW hwp AS
    SELECT /*+ BROADCAST(periods) */ * FROM periods JOIN hw ON TRUE;